In [1]:
import requests
import matplotlib.pyplot as plt
import csv
import pandas as pd
import plotly.express as px


def writeToFile(papers_by_year, save_path):
    field_names = ["Year", "NumPapers"]
    with open(save_path, "w+") as file:
        csvWriter = csv.writer(file, delimiter=",")
        csvWriter.writerow(field_names)
        for key in papers_by_year.keys():
            file.write("%s, %s\n" % (key, papers_by_year[key]))


def fetchDeepfakeAndEthicsPapers():
    papers_by_year = {}
    for year in range(2000, 2023):
        result = requests.get(
            "http://api.semanticscholar.org/graph/v1/paper/search?query=deepfake+ethic&year={}".format(
                year
            )
        )

        if result.reason != "OK":
            print("Error: ", result.status_code, result.reason)
            exit

        paper_records = result.json()
        num_papers = paper_records["total"]
        print(
            "Total number of deepfake+ethics papers in {}: {}".format(year, num_papers)
        )
        papers_by_year[year] = num_papers

    return papers_by_year


def fetchDeepfakePapers():
    papers_by_year = {}
    for year in range(2000, 2023):
        result = requests.get(
            "http://api.semanticscholar.org/graph/v1/paper/search?query=deepfake&year={}".format(
                year
            )
        )

        if result.reason != "OK":
            print("Error: ", result.status_code, result.reason)
            exit

        paper_records = result.json()
        num_papers = paper_records["total"]
        print("Total number of deepfake papers in {}: {}".format(year, num_papers))
        papers_by_year[year] = num_papers

        return papers_by_year


def plotData(file, title):
    df = pd.read_csv(file)
    fig = px.line(df, y=df.columns, x="Year", title=title)
    fig.show()

In [2]:
deepfake_file = "num_deepfake_papers_by_year.csv"
deepfake_ethics_file = "num_deepfake_ethics_papers_by_year.csv"
combined_file = "combined_data.csv"

In [ ]:
deepfake_papers_by_year = fetchDeepfakePapers()

In [ ]:
deepfake_ethics_papers_by_year = fetchDeepfakeAndEthicsPapers()

In [ ]:
writeToFile(deepfake_papers_by_year, save_path=deepfake_file)

In [ ]:
writeToFile(deepfake_ethics_papers_by_year, save_path=deepfake_ethics_file)

In [ ]:
plotData(deepfake_file, "Number of Deepfake Paper Publications Over Time")

In [ ]:
plotData(deepfake_ethics_file, "Number of Deepfake Paper Publications With Ethical Considerations Over Time")

In [ ]:
plotData(
    combined_file,
    "Number of Deepfake Paper Publications With Ethical Considerations Over Time",
)